# Imports

In [1]:
import psycopg2
from psycopg2 import DatabaseError
import logging
import requests
import datetime as dt
from parser_opennem import parse_request_json
from db_populate import insert_batch
from dateutil.relativedelta import relativedelta

# Logging

In [15]:
logger = logging.getLogger(__name__)
c_handler = logging.StreamHandler()
f_handler = logging.FileHandler('file.log')
c_handler.setLevel(logging.DEBUG)
f_handler.setLevel(logging.DEBUG)
c_format = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
c_handler.setFormatter(c_format)
f_handler.setFormatter(f_format)
logger.addHandler(c_handler)
logger.addHandler(f_handler)

# Config

In [16]:
db_name = "energy"
db_user = "dawki"
db_pword_file = "C:\\Users\\dawki\\OneDrive\\Archive\\2019\\Documents\\private.txt"
with open(db_pword_file) as f:
    db_pword = f.read()

In [17]:
from data import ZONE_KEY_TO_REGION, REGION_LONG_NAMES, NETWORK_LONG_NAMES

# Main

In [33]:
## Params
region = ZONE_KEY_TO_REGION['AUS-WA']
first_month_start = dt.date.fromisoformat('2019-01-01')
final_month_start = dt.date.today().replace(day=1)

In [34]:
## Connect to DB
dbc = psycopg2.connect(user=db_user,
                        password=db_pword,
                        host="127.0.0.1",
                        port="5432",
                        database=db_name)

In [35]:
## Teardown DB
# cur = dbc.cursor()
# with open('db_teardown.sql','r') as sql_file:
#     cur.execute(sql_file.read())
# dbc.commit()

In [36]:
## Setup DB
# cur = dbc.cursor()
# with open('db_setup.sql','r') as sql_file:
#     cur.execute(sql_file.read())
# dbc.commit()

In [37]:
while( first_month_start <= final_month_start):
    print(f"starting: {first_month_start}")
    
    ## URL
    url = f'http://api.opennem.org.au/stats/power/network/fueltech/NEM/{region}?month={first_month_start}'

    ## Request data
    logger.info(f'Requesting {url}..')
    r = requests.get(url)
    r.raise_for_status()

    ## Parse data 
    logger.info('Parsing JSON..')
    meta, price, power = parse_request_json(r.json())

    ## Set cursor
    cursor = dbc.cursor()

    ## Insert batch
    logger.info('Insert batch..')
    insert_batch(cursor,logger,meta,price,power)

    ## Commit transaction
    logger.info('Commiting to DB..')
    dbc.commit()

    first_month_start = first_month_start + relativedelta(months=1)

starting: 2019-01-01


HTTPError: 500 Server Error: Internal Server Error for url: https://api.opennem.org.au/stats/power/network/fueltech/NEM/WEM?month=2019-01-01

In [14]:
## Close connection
logger.debug('Close connection to DB..')
dbc.close()